# Criando uma RAG com Vector Search e Langchain

Notebook com demonstração de como  trabalhar com o Vector na VertexAI e criar uma RAG com Langchain, Gemini e Vector Search.

In [ ]:
# Dependências

!pip install langchain-community pypdf langchain-text-splitters langchain-google-vertexai

## Criando o Índice

In [ ]:
# Setando o projeto
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
    
BUCKET = "tutorial-index-rag"
BUCKET_URI = f"gs://{BUCKET}"

In [ ]:
# Criando o Bucket

from google.cloud import storage


def create_bucket(bucket_name, LOCATION):
    """
    Create a new bucket 
    """

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    new_bucket = storage_client.create_bucket(bucket, location=LOCATION)

    print(f'Bucket {new_bucket.name} criado')

create_bucket(BUCKET, LOCATION)

In [ ]:
# Inicializando a AI Platform

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# Criando o Índice

'''
Um índice de lote é para quando você deseja atualizar seu índice em lote, com dados que foram armazenados durante um determinado período de tempo, 
como sistemas que são processados semanalmente ou mensalmente. Um índice de streaming é quando você deseja que os dados do índice sejam atualizados 
à medida que novos dados são adicionados ao seu armazenamento de dados. O tipo que você escolhe é importante, pois a configuração e os requisitos são diferentes.
'''

my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name="embedvector-tutorial-index",
    dimensions=768,
    approximate_neighbors_count=20,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    index_update_method="STREAM_UPDATE",  # BATCH_UPDATE , STREAM_UPDATE
)

In [ ]:
# Criando o Endpoint do Índice

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="embedvector-tutorial-index-endpoint",
    public_endpoint_enabled=True,
)

In [ ]:
# Realizando o Deploy

DEPLOYED_INDEX_ID = "embedvector_tutorial_deployed"
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

In [ ]:
print (my_index.name)
print (my_index_endpoint.name)

## Geração de Embeddings do PDF

In [ ]:
# Índice
my_index = aiplatform.MatchingEngineIndex(index_name=my_index.name)

# Endpoint do Índice
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint.name)

In [ ]:
from langchain_google_vertexai import (
    VectorSearchVectorStore,
    VectorSearchVectorStoreDatastore,
)

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
def prepare_document_texts(file):

    loader = PyPDFLoader(file)
    
    documents = loader.load()
    
    text_splitter = CharacterTextSplitter(
            separator="\n", chunk_size=1000, chunk_overlap=20
    )

    docs = text_splitter.split_documents(documents)
    
    texts = []

    texts.extend(doc.page_content for doc in docs)
    
    return texts

texts = prepare_document_texts("vacinas.pdf")

In [ ]:
#Embedding documentos

from langchain_google_vertexai import VertexAIEmbeddings


embedding_model = VertexAIEmbeddings(model="text-multilingual-embedding-002")

vector_store = VectorSearchVectorStore.from_components(
    project_id=PROJECT_ID,
    region=LOCATION,
    gcs_bucket_name=BUCKET,
    index_id=my_index.name,
    endpoint_id=my_index_endpoint.name,
    embedding=embedding_model,
)

vector_store.add_texts(texts=texts, is_complete_overwrite=True)

## Busca por Similaridade

In [ ]:
# Busca por Similaridade

vector_store.similarity_search("vacina", k=1)

In [ ]:
# VectorStore como Retriever

retriever = vector_store.as_retriever()

# Pergunta
retriever.invoke("Quais são as vacinas recomendadas para idosos?")

## RAG

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_vertexai import VertexAI

In [ ]:
# Define a system prompt that tells the model how to use the retrieved context
system_prompt = """Você é um assistente para tarefas de resposta a perguntas. 
Use as seguintes partes do contexto recuperado para responder à pergunta. 
Se você não sabe a resposta, basta dizer que não sabe. 
Use no máximo três frases e mantenha a resposta concisa.
Contexto: {context}:"""

    

def retrieve_contexts(question:str)-> str: 
    # Recuperando contextos 
    docs = retriever.invoke(question)

    # Combinando os contextos 
    docs_text = "".join(d.page_content for d in docs)
    
    return docs_text


def generate_answer(question: str)-> str:
     
    # Update the model as needed
    llm = VertexAI(model_name="gemini-2.0-flash-001")
    
    contexto = retrieve_contexts(question)
    
    system_prompt_fmt = system_prompt.format(context=contexto)

    response = llm.invoke([SystemMessage(content=system_prompt_fmt),
                          HumanMessage(content=question)])
    return response

In [ ]:
### Perguntas
question = """Quais vacinas um adulto deve tomar?"""

resposta = generate_answer(question)

print(resposta)

In [ ]:
### Perguntas
question = """Quais vacinas uma criança deve tomar durante o primeiro ano de vida?"""

resposta = generate_answer(question)

print(resposta)

## Listando os Índices

In [ ]:
# Initialize the Vertex AI client
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# List Indexes
aiplatform.MatchingEngineIndex.list()

In [ ]:
# List Indexes
aiplatform.MatchingEngineIndexEndpoint.list()

### Limpando o Índice e Bucket na Google Cloud (Opcional)

In [ ]:
# Desfazer Deploy do Índice
my_index_endpoint.undeploy_index(deployed_index_id=DEPLOYED_INDEX_ID)

In [ ]:
input("Pressione Enter para deletar o Endpoint do Índice e o Índice:")

# deletar Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# deletar Index
my_index.delete()

In [ ]:
# Apagar Cloud Storage bucket
! gcloud storage rm {BUCKET_URI} --recursive

Referências

https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index?hl=pt-br#create-index-batch

https://python.langchain.com/v0.1/docs/integrations/vectorstores/google_vertex_ai_vector_search/

https://medium.com/google-cloud-brasil/rag-super-customizado-com-vertex-ai-vector-search-e-langchain-53f2d7a8d4b8

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb

#VertexAISprint